In [145]:
#challenege 1 cross validation sgd \r-> escape 문이다 멍충아!!!
def createCVBatFileForLibFM(dim='30',iter='1000',regular='0,0,1',method='mcmc'):
    for i in range(10):
        with open('bat/train'+str(i)+'.bat','w') as fw:
            fw.write("..\\libfm -task r -train ..\\cv\\cvTrain"+str(i)+".libfm -test ..\\cv\\cvTest"+str(i)+".libfm -dim '1,1,")
            if method=='sgd':
                fw.write(dim+"' -iter "+iter+" -method sgd -learn_rate 0.00001 -regular "+regular+" -meta ..\\libfmInput\\meta.txt")
            elif method=='mcmc':
                fw.write(dim+"' -iter "+iter+" -meta ..\\libfmInput\\meta.txt")
            fw.write(" -init_stdev 0.1 -out ..\\prediction\\result"+str(i))
            
createCVBatFileForLibFM(iter='1000',regular='0,0.01,1',method='sgd')

In [ ]:
#1024개 단위로 write하기 위한 algorithm
tmp=100000
shift_num=10
bit=0

for i in range(tmp+1):
    if (i>>shift_num)!=bit:
        print i
        bit+=1
        print bit

In [136]:
lassoPolyFeature={}
libfmId=0
groupData=[] #libfm option중 group id list
groupId=0 #group id 값
featureLen=0

with open('tmp/df0.csv','r') as fr:
    lines=fr.readlines()
    featureLen=len(lines[0].strip().split(',')[1:])
    
    for line in lines[1:]:
        feature=line.strip().split(',')
        result=''
        for i in range(len(feature[1:])):
            if feature[i+1]: result+=str(i+libfmId)+':'+str(feature[i+1])+' '
        lassoPolyFeature[int(feature[0])]=result

libfmId+=featureLen
groupData.extend([str(groupId)]*featureLen)
groupId+=1

In [83]:
df=pd.read_csv('tmp/lasso_polyfeatures.csv',index_col=0)
idPolyFeature={}
for _ in df.index:
    feature=''
    for poly in df.columns:
        if df.loc[_,poly]:feature+=str(int(poly)+libfmId)+':'+str(df.loc[_,poly])+' '
    idPolyFeature[_]=feature

{0: '122:1.0 123:1.0 124:1.0 125:-1.0',
 1: '48:-1.0',
 2: '27:1.0 159:1.0 160:-1.0 161:1.0',
 3: '129:-1.0 130:-1.0 131:-1.0 145:-1.0 146:1.0 147:-1.0 148:-1.0 149:-1.0 157:1.0 158:1.0',
 4: '14:1.0 26:1.0 74:1.0 75:-1.0 175:-1.0 176:-1.0 177:1.0 178:-1.0 179:1.0',
 5: '',
 6: '175:-1.0 176:-1.0 177:-1.0 178:-1.0 179:-1.0',
 7: '96:-1.0 97:-1.0',
 8: '145:-1.0 146:-1.0 147:-1.0 148:-1.0 149:-1.0 157:1.0 158:-1.0',
 9: '55:-1.0 70:-1.0 71:-1.0 72:-1.0',
 10: '',
 11: '55:-1.0 58:-1.0 59:-1.0 60:-1.0 61:-1.0 62:-1.0 63:-1.0 67:-1.0 68:-1.0 69:-1.0 70:-1.0 71:1.0 72:1.0',
 12: '96:-1.0 97:-1.0 115:-1.0 116:-1.0 117:-1.0',
 13: '143:-1.0 144:-1.0 145:-1.0 146:-1.0 147:-1.0 148:-1.0 149:-1.0 157:1.0 158:1.0',
 14: '',
 15: '28:1.0 29:1.0 55:-1.0 70:-1.0 71:1.0 72:1.0',
 16: '143:-1.0 144:1.0 175:-1.0 176:-1.0 177:-1.0 178:1.0 179:-1.0',
 17: '145:-1.0 146:-1.0 147:1.0 148:1.0 149:-1.0 157:1.0 158:1.0',
 18: '55:-1.0 70:-1.0 71:-1.0 72:-1.0 144:-1.0',
 19: '',
 20: '145:-1.0 146:-1.0 147:-1

In [ ]:
pr=pd.read_csv('drug/Drug_info_release_extended.csv')
dtl=set() # drug target list
for targets in pr['Target']:
    if pd.notnull(targets):
        targetSplit=targets.split(',')
        for i in range(len(targetSplit)):
            if '*' not in targetSplit[i]:
                dtl.add(targetSplit[i].strip())

df=pd.read_csv('gex/gex.csv',index_col=0)
dtGexl=[]
for dt in dtl:
    if dt in df.index: dtGexl.append(dt)
        
with open('gex/drugTargetGex.txt','w') as fw:
    for _ in dtGexl:
        fw.write(_+'\n')

In [42]:
import pandas as pd
libfmId=0
groupId=0
gexBool=1
groupData=[]
if gexBool:
    dtl=[]
    with open('gex/drugTargetGex.txt','r') as fr:
        for line in fr:
            dtl.append(line.strip())

    gexId={}
    for i,d in enumerate(dtl,start=libfmId):
        gexId[d]=i
        groupData.append(str(groupId))    

    libfmId+=len(gexId)
    print 'gex',len(gexId)
    groupId+=1

    df=pd.read_csv('gex/gex.csv',index_col=0)
    df=df.loc[dtl,:]
    clGex={}
    for gene in df.index:
        std=df.loc[gene,:].std()
        mean=df.loc[gene,:].mean()
        for cellLine in df.columns:
            if cellLine not in clGex: clGex[cellLine]=''
            stdCoef=1 #이걸 바꿔주면서 실험해보면 어떨까, 0이면 그냥 양 음 체크
            #양의 값
            if df.loc[gene,cellLine]>=(mean+stdCoef*std):clGex[cellLine]+=str(gexId[gene])+':'+str((df.loc[gene,cellLine]-mean)/std)+' '
            #음의 값
            if df.loc[gene,cellLine]<=(mean-stdCoef*std):clGex[cellLine]+=str(gexId[gene])+':'+str((df.loc[gene,cellLine]-mean)/std)+' '

gex 72
{'BCL2L1': 0, 'TNKS': 18, 'PDGFRA': 1, 'TNFSF10': 19, 'RRM1': 61, 'MAP2K1': 60, 'TYMS': 68, 'BIRC5': 54, 'ATM': 33, 'BIRC3': 47, 'HDAC3': 32, 'RET': 34, 'PRKDC': 35, 'MAP2': 58, 'AKT1': 2, 'AKT2': 3, 'AKT3': 4, 'ATR': 37, 'KIF11': 38, 'MAP4': 53, 'ADAM17': 62, 'FLT4': 39, 'RAF1': 40, 'FLT1': 41, 'TEK': 42, 'MET': 51, 'PIK3CA': 5, 'PIK3CB': 6, 'DNMT1': 7, 'PIK3CD': 8, 'TTK': 20, 'JAK2': 21, 'JAK1': 22, 'FASN': 43, 'WEE1': 9, 'BRAF': 69, 'KDR': 45, 'CHEK1': 23, 'SRC': 46, 'FNTA': 10, 'PIM1': 64, 'EGFR': 55, 'PARP6': 24, 'TOP1MT': 48, 'PARP1': 25, 'TUBB': 57, 'CHUK': 26, 'AR': 65, 'MTOR': 49, 'UBA3': 27, 'NAE1': 44, 'FGFR4': 11, 'TUBB1': 28, 'FGFR2': 12, 'FGFR3': 13, 'CSNK2A1': 14, 'FGFR1': 15, 'ALK': 50, 'ESR1': 29, 'SYK': 16, 'SLC16A3': 66, 'SLC16A4': 67, 'XIAP': 36, 'TOP1': 30, 'MAP2K2': 59, 'NR1I2': 17, 'HDAC1': 31, 'MAPT': 70, 'CMPK1': 63, 'BIRC2': 56, 'HSP90AA1': 52, 'BCL2': 71}


In [ ]:
#cnv
import pandas as pd
import numpy as np
df=pd.read_csv('cnv/cnv_gene.csv',usecols=[0,1],dtype={'cell_line_name':str,'gene':str},engine='c')
grouped=df.groupby('gene')
for name,group in grouped:
    print name
    print group

39340
   cell_line_name   gene
0           22RV1  39340
1           647-V  39340
2            A549  39340
3        BFTC-905  39340
4           BT-20  39340
5          BT-474  39340
6          BT-549  39340
7             C32  39340
8         CAL-120  39340
9         CAL-148  39340
10         CAL-51  39340
11         CAMA-1  39340
12       COLO-205  39340
13         Calu-3  39340
14         Calu-6  39340
15        DMS-114  39340
16        DU-4475  39340
17         EVSA-T  39340
18        HCC1143  39340
19        HCC1187  39340
20        HCC1395  39340
21        HCC1419  39340
22        HCC1428  39340
23        HCC1500  39340
24        HCC1569  39340
25        HCC1806  39340
26        HCC1937  39340
27        HCC1954  39340
28          HCC38  39340
29          HCC70  39340
..            ...    ...
55      NCI-H1793  39340
56      NCI-H1975  39340
57      NCI-H2085  39340
58      NCI-H2170  39340
59      NCI-H2228  39340
60       NCI-H226  39340
61      NCI-H2291  39340
62        NCI-H23  

In [57]:
## mutation에 얼마나 많은 drug target을 포함하고 있는가 출력해보는 코드
## 각 cell line에서 unique하게 drug target gene이 한번이라도 있으면 gene을 set에 포함하였다. 즉, 여러 point mutation이라도 한번으로 셈하였다.
## BRAF_V600E는 point mutation으로 셈하였다.

import pandas as pd
pr=pd.read_csv('drug/notInMut.csv')
dtl=set() # drug target list
for targets in pr['Target']:
    if pd.notnull(targets):
        targetSplit=targets.split(',')
        for i in range(len(targetSplit)):
            if '*' not in targetSplit[i]:
                dtl.add(targetSplit[i].strip())
                
clMut={}
pr=pd.read_csv('mutation/mutations.csv')

#---------------------------------------------------------------------------------------
#metastasis는 tumor gene이 drug target에 포함 되어도 mutation feature로 포함하지 않았다
#---------------------------------------------------------------------------------------
for row in pr[pr['Tumour.origin']=='primary'].itertuples():
    if row[1] in dtl: 
        if row[1]=='BRAF' and row[15]=='p.V600E':
            if row[5] in clMut: clMut[row[5]].add('BRAF_V600E')
            else: clMut[row[5]]=set(['BRAF_V600E'])
        else:
            if row[5] in clMut: 
                clMut[row[5]].add(row[1])
            else: clMut[row[5]]=set([row[1]])

geneName=[]
for mutationGenes in clMut.values():
    for item in mutationGenes: geneName.append(item)
    
geneNameCount={}
for x in dtl:
    geneNameCount[x]=geneName.count(x)
    
import matplotlib.pyplot as plt
plt.bar(range(len(geneNameCount)), geneNameCount.values(),align='center')
plt.xticks(range(len(geneNameCount)), geneNameCount.keys(),rotation=90)

plt.show()

In [5]:
import pandas as pd
pr=pd.read_csv('mutation/mutations.csv')
for item in pr.columns:
    print item

Gene.name
Accession.Number
Gene.CDS.length
HGNC.ID
cell_line_name
ID_sample
ID_tumour
Primary.site
Site.subtype
Primary.histology
Histology.subtype
Genome.wide.screen
Mutation.ID
Mutation.CDS
Mutation.AA
Mutation.Description
Mutation.zygosity
GRCh
Mutation.genome.position
strand
SNP
FATHMM.prediction
Mutation.somatic.status
Pubmed_PMID
ID_STUDY
Institute
Institute.Address
Catalogue.Number
Sample.source
Tumour.origin
Age
Comments


In [15]:
comments=set(pr['Comments'])
for item in comments:
    if 'Stage:' in str(item):
        print item

Stage:IV
Stage:III
Stage:III,Metastatic site:NA
Grade:II,Stage:III
Grade:IV,Metastatic site:lymph node,Stage:III
Stage:I
Grade:II,Stage:II
Grade:II,Stage:IV
Stage:II
Grade:III,Metastatic site:lymph node,Stage:III


In [59]:
cellLineName=set(pr['cell_line_name'])
for x in cellLineName:
    comments=set(pr.loc[pr['cell_line_name']==x,'Comments'])
    for y in comments:
        if 'Stage:' in str(y): print x,y

HCC1419 Grade:III,Metastatic site:lymph node,Stage:III
SW900 Stage:IV
HCC1806 Grade:II,Stage:II
UACC-812 Grade:II,Stage:IV
HCC38 Grade:II,Stage:III
HCC70 Grade:III,Metastatic site:lymph node,Stage:III
HCC1143 Stage:III
HT-1197 Stage:IV
MFM-223 Stage:III,Metastatic site:NA
HT-1376 Stage:III
647-V Stage:II
HCC1937 Stage:III
SW780 Stage:I
SW837 Stage:IV
HCC1954 Stage:III
HCC1569 Grade:IV,Metastatic site:lymph node,Stage:III
KU-19-19 Stage:III
HCC1187 Stage:III
SW948 Stage:III


In [3]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 02 13:56:55 2015

@author: heewon
"""
import random as rd

for idx in range(0,100):
    df=open('input/ch1_train_combination_and_monotherapy.csv','r')
    drugdic={}
    i=0
    for line in df:
        if 'CELL_LINE' in line:
            cind=line
            continue
        #checking the QA score
        row=line.split(',')
        if row[12]=='1':
            if drugdic.has_key(row[13])==False:
                drugdic[row[13]]=[]
            l=drugdic.get(row[13])
            l.append(line)
            drugdic[row[13]]=l
            i+=1
    df.close()
    shufflelist=[]
    df1=open('input/chall1/chall1_train_set'+str(idx)+'.csv','w')
    df2=open('input/chall1/chall1_test_set'+str(idx)+'.csv','w')
    df1.write(cind)
    df2.write(cind)
    #print cind
    lenkey=len(drugdic.keys())
    for key in drugdic.keys():
        l=drugdic.get(key)
        tg=rd.choice(l)
        l.remove(tg)
        df1.write(tg)
        shufflelist.extend(l)
        
        
    rd.shuffle(shufflelist)
    for l in shufflelist:
        #print l
        if shufflelist.index(l)<(i/10*8-lenkey-40):
            df1.write(l)
        else:
            df2.write(l)

    df1.close()
    df2.close()

In [61]:
if lipinskiBool:
    pr=pd.read_csv('drug/drug_lipinski.csv')
    drugLipinski={}
    maxLipinski=int(max(pr.loc[pr.Lipinski.notnull(),'Lipinski']))
    
    for row in pr[pr.Lipinski.notnull()].itertuples():
        drugLipinski[row[1]]=str(libfmId+int(row[2]))+':1'

    libfmId+=maxLipinski
    groupData.extend([str(groupId)]*maxLipinski)
    groupId+=1
    print 'lipinski',maxLipinski

lipinski 3


In [ ]:
if stageBool:
    pr=pd.read_csv('cell/cell_comment_stage.csv')
    clStage={}
    maxStage=max(pr['stage'])
    #print maxStage
    for row in pr.itertuples():
        #row[1] == cell line name, row[2] == stage
        clStage[row[1]]=str(libfmId+row[2]-1)+':1'

    libfmId+=maxStage
    groupData.extend([str(groupId)]*maxStage)
    groupId+=1
    print 'cell comment stage',maxStage

In [2]:
#random shuffle 연습
#input = ch1 train monotherapy
#output = ch1 train monotherapy qa1 shuffle1 
import numpy as np
import pandas as pd
pr=pd.read_csv('input/ch1_train_combination_and_monotherapy.csv')
pr=pr.loc[pr.QA==1,:]
prShuffled=pr.reindex(np.random.permutation(pr.index)).copy()
#prShuffled.to_csv('input/ch1_train_combination_and_monoTherapy_qa1_shuffle5.csv',index=False)

       CELL_LINE   COMPOUND_A     COMPOUND_B  MAX_CONC_A  MAX_CONC_B  \
168      HCC1428          AKT        PIK3C_2        1.00         1.0   
854   MDA-MB-231         EGFR        MAP2K_1        1.00         0.1   
1128     HCC1937          AKT           FGFR        1.00         1.0   
997   MDA-MB-231        AKT_1           ESR1        1.00         1.0   
1203     CAL-120         FGFR     PIK3C_MTOR        1.00         3.0   
1323    COLO-205        CHEK1            IAP        1.00         1.0   
983        HCC38         ERBB       PIK3CA_4        1.00         1.0   
1916  MDA-MB-231  BCL2_BCL2L1           PIM1        1.00         1.0   
351        HCC70        ATR_4    BCL2_BCL2L1        1.00         1.0   
85         HCC38       ADAM17  PIK3CB_PIK3CD        1.00        75.0   
1718    Hs-578-T        PARP1          PIK3C        1.00         3.0   
1675     HCC1395      MAP2K_1         MTOR_1        1.00         1.0   
740    NCI-H2291   CarboTaxol      CSNK2A1_2        0.01        

In [ ]:
monotherapydf=pd.read_csv('input/monotherapy_integrated.csv')
compoundVarA='BRAF_M2'
compoundVarB='MAP2K_1'
monotherapydf.loc[(monotherapydf['COMPOUND_A']==compoundVarA)&(monotherapydf['COMPOUND_B']==compoundVarB)
                  ,('CELL_LINE','COMPOUND_A','COMPOUND_B','SYNERGY_SCORE')]

In [ ]:
mutationdf=pd.read_csv('mutation/mutations.csv')
#synergy
celllinevar='MDA-MB-468'
celllinevar1='A549'
celllinevar2='KATOIII'
#additive
celllinevar3='EVSA-T'
celllinevar4='HT-1376'

mutationGeneName=set(mutationdf.loc[mutationdf['cell_line_name']==celllinevar,'Gene.name'])
mutationGeneName1=set(mutationdf.loc[mutationdf['cell_line_name']==celllinevar1,'Gene.name'])
mutationGeneName2=set(mutationdf.loc[mutationdf['cell_line_name']==celllinevar2,'Gene.name'])

mutationGeneName3=set(mutationdf.loc[mutationdf['cell_line_name']==celllinevar3,'Gene.name'])
mutationGeneName4=set(mutationdf.loc[mutationdf['cell_line_name']==celllinevar4,'Gene.name'])
synergisticGenes=mutationGeneName.intersection(mutationGeneName1)
#mutationGeneName.intersection(mutationGeneName1,mutationGeneName2)
print 'synergistic genes',synergisticGenes,'\n'

#with open('tmp/NIC-SNU-16+T-24.txt','w') as fw:
#    for item in synergisticGenes:
#        fw.write(item+'\n')

synergyExcludeAdditive=synergisticGenes.difference(mutationGeneName3)

print synergyExcludeAdditive


mutationdf.loc[(mutationdf['cell_line_name'].isin([celllinevar,celllinevar1]))&(mutationdf['Gene.name'].isin(synergyExcludeAdditive))
               ,('Gene.name','cell_line_name','Primary.site','Site.subtype','Primary.histology','Histology.subtype','Mutation.CDS','Mutation.AA'
                 ,'Mutation.Description','Mutation.zygosity','Mutation.genome.position','strand','SNP','Tumour.origin'
                 ,'Comments')].to_csv('tmp/'+celllinevar+'+'+celllinevar1+'--'+celllinevar3+'.csv',index=False)

In [ ]:
pr=pd.read_csv('methyl/CpG_isle_level/methyl_ilse_beta.csv',index_col=0)
methylSiteSet=set()
for colName in pr.columns.values:
    tmp=pr.sort(colName,ascending=False).head(10).index.values.tolist()
    for methylVal in tmp:
        methylSiteSet.add(methylVal)

In [ ]:
testDir='input/ch1_leaderBoard_monoTherapy_blankErased.csv'
pr=pd.read_csv('input/ch1_train_combination_and_monoTherapy.csv')
for x in set(pr['COMBINATION_ID']):
    print x,
    x=x.split('.')
    u=set(ddt[x[0]])|set(ddt[x[1]])

In [ ]:
pr=pd.read_csv('mutation/mutations.csv')
geneNameSet=set()
for geneName in pr['Gene.name']:
    geneName=geneName.split('_')[0]
    geneNameSet.add(geneName)
    
geneNameSetLen=len(geneNameSet)
geneNameList=list(geneNameSet)

for i in range(geneNameSetLen):
    if i%1000==0:
        print '\n',i
        print geneNameList[i],
    else: print geneNameList[i],

Breast
       Sanger.Name Disease.Area
4            BT-20       Breast
5           BT-474       Breast
6           BT-549       Breast
8          CAL-120       Breast
9          CAL-148       Breast
10          CAL-51       Breast
13          CAMA-1       Breast
16         DU-4475       Breast
17          EVSA-T       Breast
18         HCC1143       Breast
19         HCC1187       Breast
20         HCC1395       Breast
21         HCC1419       Breast
22         HCC1428       Breast
23         HCC1500       Breast
24         HCC1569       Breast
25         HCC1806       Breast
26         HCC1937       Breast
27         HCC1954       Breast
28           HCC38       Breast
29           HCC70       Breast
31        Hs-578-T       Breast
41            MCF7       Breast
42      MDA-MB-157       Breast
43  MDA-MB-175-VII       Breast
44      MDA-MB-231       Breast
45      MDA-MB-361       Breast
46      MDA-MB-415       Breast
47      MDA-MB-436       Breast
48      MDA-MB-453       Breast
4

In [4]:
#drug target에 있는 target들의 gene expression 탐구 중...

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

pr=pd.read_csv('drug/Drug_Targets_Extended_by_the_document.csv')
dtl=set() # drug target list
for targets in pr['Target']:
    if pd.notnull(targets):
        targetSplit=targets.split(',')
        for i in range(len(targetSplit)):
            if '*' not in targetSplit[i]:
                dtl.add(targetSplit[i].strip())

pr=pd.read_csv('cell/cell_info.csv')
pr=pr[['Sanger.Name','Disease.Area']]
grouped=pr.groupby('Disease.Area')
sangerName=[]
for name,group in grouped:
    #print name
    #print group
    #sangerName.extend(group['Sanger.Name'].tolist())
    
    group=[x for x in group['Sanger.Name'].tolist() if x not in ['MDA-MB-175-VII','NCI-H1437']] #gene expression이 없다
    
    pr=pd.read_csv('gex/gex.csv',index_col=0)
    pr=pr[group] #re-order the cell line columns by disease area
    stdByGene=pr.std(axis=1)
    globalStd=stdByGene.std()
    pr=pr.loc[pr.index.isin(dtl)]
    looseRestriction=0.5
    smallerThanGlobalStdGenes=pr.std(axis=1)<=globalStd+looseRestriction
    #pr[smallerThanGlobalStdGenes].to_csv('gex/'+name+'_drug_target_gex_restricted_by_global_std+'+str(looseRestriction)+'.csv')
    #pr[smallerThanGlobalStdGenes].plot(kind='bar',title=name)
    #pr.plot(kind='bar',title=name)
    #plt.show()

In [88]:
#point mutation 개수 셈하여 1 value대신 point mutation 개수를 value로 줌

from collections import defaultdict
libfmId=0
pr=pd.read_csv('drug/notInMut.csv')
drugTargetSetTmp=set() # drug target list
for targets in pr['Target']:
    if pd.notnull(targets):
        targetSplit=targets.split(',')
        for i in range(len(targetSplit)):
            if '*' not in targetSplit[i]:
                drugTargetSetTmp.add(targetSplit[i].strip())

dtId2={}
for i,d0 in enumerate(drugTargetSetTmp,start=libfmId):
    dtId2[d0]=i

libfmId+=len(dtId2)

clMut={}
pr=pd.read_csv('mutation/mutations.csv')

#---------------------------------------------------------------------------------------
#metastasis는 tumor gene이 drug target에 포함 되어도 mutation feature로 포함하지 않았다
#---------------------------------------------------------------------------------------
clMut={}
for targets in drugTargetSetTmp:
    if targets=='BRAF_V600E':
        tmp=pr.loc[(pr['Gene.name']=='BRAF')&(pr['Tumour.origin']=='primary')&(pr['Mutation.AA']=='p.V600E'),'cell_line_name'].tolist()
        for cellLine in set(tmp):
            if cellLine not in clMut: clMut[cellLine]={} #초기화
            clMut[cellLine][dtId2[targets]]=tmp.count(cellLine) #point mutation 개수
    else:
        tmp=pr.loc[(pr['Gene.name']==targets)&(pr['Tumour.origin']=='primary'),'cell_line_name'].tolist()
        for cellLine in set(tmp):
            if cellLine not in clMut: clMut[cellLine]={} #초기화
            clMut[cellLine][dtId2[targets]]=tmp.count(cellLine) #point mutation 개수

#drug target 리스트 내 gene 중 cell line mutation 중 그 gene이 없는 경우 -1로 줌
clNotInMut=defaultdict(set)
for cellLineVar in clMut:
    for drugTargetIdVar in dtId2.values():
        if drugTargetIdVar not in clMut[cellLineVar]:
            clNotInMut[cellLineVar].add(drugTargetIdVar)

In [33]:
#2 row씩 뛰면서 출력
pr=pd.read_csv('input/chall1/ch1_leaderBoard_monoTherapy.csv')
pr[1::2].to_csv('input/chall1/ch1_leaderBoard_monoTherapy_noblank.csv',index=False)